# **授業課題としてTelegram上で動作されるデモチャットボット（対話システム）を作成しました。この対話システムはあなたのカウンセラーになるように促されています**

In [1]:
!pip install langchain langchain_community accelerate bitsandbytes sentence_transformers
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/

まず、Telegramをインストールする必要があります。このチャットボットはあなたのカウンセラーであり、どんな問題でも相談することができます。

In [ ]:
import sys
!{sys.executable} -m pip install python-telegram-bot==12.8
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import os
import signal

# アクセストークン（先ほど発行されたアクセストークンに書き換えてください）
TOKEN = "**********************************" #自分のトークンを入力  #秘密漏洩のリスクがあるためトークンを削除しました
    #トークンを発行する方法:　1.Telegramの検索欄に「BotFather」と入力 2.BotFatherを選択し,「START」をクリック 3. メッセージ欄に「/newbot」を送信　
    # 4. 好きな名前(今回はtushig_bot)を入力して新しいbotを作成 5. アクセストークンが発行される

class TelegramBot:
    def __init__(self, system):
        self.system = system

    def start(self, bot, update):
        # 辞書型 inputにユーザIDを設定
        input = {'utt': None, 'sessionId': str(update.message.from_user.id)}

        # システムからの最初の発話をinitial_messageから取得し，送信
        update.message.reply_text(self.system.initial_message(input)["utt"])

    def message(self, bot, update):
        # 辞書型 inputにユーザからの発話とユーザIDを設定
        input = {'utt': update.message.text, 'sessionId': str(update.message.from_user.id)}

        # replyメソッドによりinputから発話を生成
        system_output = self.system.reply(input)

        # 発話を送信
        update.message.reply_text(system_output["utt"])

    def end(self, bot, update):
      os.kill(os.getpid(), signal.SIGINT)

    def run(self):
        updater = Updater(TOKEN)
        dp = updater.dispatcher
        dp.add_handler(CommandHandler("start", self.start))
        dp.add_handler(CommandHandler("end", self.end))
        dp.add_handler(MessageHandler(Filters.text, self.message))
        updater.start_polling()
        updater.idle()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.4/375.4 kB 9.1 MB/s eta 0:00:00


Chatgpt 3.5 turbo モデルを使用。Telegram上で「tushig_bot」というアカウント名を検索し、chatbot(@tushig_bot)を選択し、「START」をクリックし、対話を開始する。「/end」コマンドで対話を終了する。

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.chat_models import ChatOpenAI

class LLMSystem:
  def __init__(self):
    self.contexts = ""
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=128,openai_api_key="***********************************") #秘密漏洩のリスクのためAPI_keyを削除しました
    # LLMの準備

    # プロンプト
    template = "# You are psychologist. You ask questions about user's feeling. Keep conversation going by asking questions. You can give them proper advises based on user's situation. You can give them motivation and encouragement. Have empathy for them. Do not give them too long answer and just keep it as a simple conversation. Also, you can recommend them some way to cope with their problem. ¥n¥n{contexts}AI:"
    prompt = PromptTemplate(template=template, input_variables=["contexts"])
    self.llm_chain = LLMChain(prompt=prompt, llm=llm)

  def initial_message(self, input):
    self.contexts = "AI: Hello, lets staenrt a conversation。\n"
    return {'utt': 'Hello, lets start a conversation。', 'end':False}

  def reply(self, input):
    self.contexts += f"ユーザ:{input['utt']}¥n"
    response = self.llm_chain.run(question=input['utt'], contexts=self.contexts)
    print(response)
    response = response.split(self.contexts)[-1].split("¥n")[0].replace("AI:", "")
    self.contexts += f"AI:{response}¥n"
    return {"utt": response, "end": False}

if __name__ == '__main__':
  system = LLMSystem()
  bot = TelegramBot(system)
  bot.run()

<ipython-input-3-ec1e411c1ea9>:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=128,openai_api_key="sk-None-xahzF3Tg0YeO85a9U2RRT3BlbkFJ2U1Lp7FYgt5LMH6MWPOH")
<ipython-input-3-ec1e411c1ea9>:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.llm_chain = LLMChain(prompt=prompt, llm=llm)
<ipython-input-2-1ae053b58eb6>:36: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details
  updater = Updater(TOKEN)
<ipython-input-3-ec1e411c1ea9>:22: LangChainDeprecationWarning

Hi there! How are you feeling today?
I'm sorry to hear that you're feeling down. Can you tell me more about what's been bothering you lately?


# **結果**
動作例はフォルダ中のpsychologist(counsellor)_chatbot pdfファイルの中から見ることができます。